# TP4 - DÉTECTION DE FRAUDE BANCAIRE
## Optimisation ML & Feature Engineering Avancé

**Master 1 Data Engineering - Concepts & Techno IA** 
**Durée : 7 heures** 
**Nom & Prénom : ___________________________**

---

### Objectifs
- Maîtriser le Feature Engineering avancé
- Gérer des données fortement déséquilibrées (0.17% de fraudes)
- Optimiser des hyperparamètres avec GridSearchCV
- Construire des Pipelines ML complets
- Analyser la performance (ROC, Learning Curves, Feature Importance)

### Mission
Développer un modèle de détection de fraude avec :
- **Recall ≥ 0.85** (détecter 85% des fraudes)
- **Precision maximale** (minimiser les faux positifs)
- **Explicabilité** (Feature Importance)

---

## IMPORTS & CONFIGURATION

In [ ]:
# Imports standards
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuration visualisation
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Reproductibilité
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print(" Imports réussis")

In [ ]:
# Imports ML
from sklearn.model_selection import (
 train_test_split, StratifiedKFold, TimeSeriesSplit,
 GridSearchCV, cross_val_score, learning_curve
)
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline

# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Métriques
from sklearn.metrics import (
 classification_report, confusion_matrix, ConfusionMatrixDisplay,
 roc_auc_score, average_precision_score, precision_recall_curve,
 roc_curve, auc, f1_score, precision_score, recall_score
)

# Gestion déséquilibre
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

print(" Imports ML réussis")

---
## PARTIE 1 : EXPLORATION & FEATURE ENGINEERING (2h)
---

### 1.1 Chargement des Données

In [ ]:
# TODO: Charger le dataset creditcard.csv depuis le dossier ../data/

df = None # À compléter

# Afficher les informations
print(f"Shape: {df.shape}")
df.head()

In [ ]:
# TODO: Afficher les informations du dataset



### 1.2 Analyse du Déséquilibre

In [ ]:
# TODO: Calculer et afficher :
# - Nombre de transactions légitimes (Class = 0)
# - Nombre de fraudes (Class = 1)
# - Pourcentage de fraudes
# - Ratio fraudes/légitimes

fraud_count = None # À compléter
legit_count = None # À compléter
fraud_percentage = None # À compléter

print(f"Transactions légitimes: {legit_count:,}")
print(f"Fraudes: {fraud_count:,}")
print(f"Pourcentage de fraudes: {fraud_percentage:.3f}%")
print(f"Ratio (1 fraude pour X légitimes): 1:{legit_count/fraud_count:.0f}")

In [ ]:
# TODO: Visualiser la distribution des classes avec un bar plot



### QUESTION 1
**Analysez les résultats ci-dessus et répondez :**

1. Quel est le ratio fraudes/légitimes ? Est-ce un déséquilibre important ?
2. Pourquoi ce déséquilibre est-il un problème pour le Machine Learning ?
3. Quelles techniques pouvez-vous utiliser pour gérer ce déséquilibre ?

**Vos réponses :**

```
1. ...

2. ...

3. ...
```

### 1.3 Analyse des Distributions

In [ ]:
# TODO: Comparer les statistiques (mean, std, min, max) pour Amount et Time
# entre fraudes et transactions légitimes

print("="*60)
print("TRANSACTIONS LÉGITIMES")
print("="*60)
# À compléter

print("\n" + "="*60)
print("FRAUDES")
print("="*60)
# À compléter

In [ ]:
# TODO: Créer 2 histogrammes côte à côte pour visualiser la distribution de Amount
# Un pour les légitimes, un pour les fraudes

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# À compléter

plt.tight_layout()
plt.show()

In [ ]:
# TODO: Créer des boxplots pour comparer Amount entre fraudes et légitimes



### 1.4 Analyse de Corrélation

In [ ]:
# TODO: Calculer la corrélation de toutes les features avec 'Class'
# Afficher les 10 features les plus corrélées (en valeur absolue)

correlations = None # À compléter

print("Top 10 Features corrélées avec Class:")
# À compléter

In [ ]:
# TODO: Créer une heatmap des 10 features les plus corrélées avec Class



### QUESTION 2
**Analysez les corrélations :**

1. Quelles sont les 3 features PCA les plus corrélées avec Class ?
2. Les fraudes ont-elles des montants typiques différents des transactions légitimes ?
3. Y a-t-il des patterns temporels visibles ?

**Vos réponses :**

```
1. ...

2. ...

3. ...
```

### 1.4bis Analyse Temporelle Approfondie

In [ ]:
# TODO: Analyser la distribution des fraudes par heure
# Créer un graphique montrant le nombre de fraudes par heure
# Comparer avec les transactions légitimes



In [ ]:
# TODO: Analyser les patterns jour/nuit
# Calculer le taux de fraude pour chaque heure
# Formule: taux_fraude(h) = nb_fraudes(h) / nb_total_transactions(h)
# Identifier les heures à risque élevé



In [ ]:
# TODO: Créer une matrice de corrélation entre Time, Amount et les top 5 features PCA
# Visualiser avec une heatmap



---
### 1.5 Feature Engineering Avancé
**Mission : Créer au minimum 15 nouvelles features pertinentes**

In [ ]:
# Créer une copie pour le feature engineering
df_fe = df.copy()

print(f"Shape initiale: {df_fe.shape}")

#### Features Temporelles

In [ ]:
# TODO: Créer les features temporelles suivantes :
# 1. 'hour' : Heure de la journée (0-23) à partir de Time
# 2. 'day' : Jour (0 ou 1) à partir de Time
# 3. 'hour_sin' : Encodage cyclique sin de l'heure
# 4. 'hour_cos' : Encodage cyclique cos de l'heure

df_fe['hour'] = None # À compléter
df_fe['day'] = None # À compléter
df_fe['hour_sin'] = None # À compléter
df_fe['hour_cos'] = None # À compléter

print(" Features temporelles créées")
df_fe[['Time', 'hour', 'day', 'hour_sin', 'hour_cos']].head()

In [ ]:
# TODO: Créer une feature 'period' (période de la journée)
# Nuit: 0-6h, Matin: 6-12h, Après-midi: 12-18h, Soir: 18-24h

def get_period(hour):
 """Retourne la période de la journée"""
 # À compléter
 pass

df_fe['period'] = None # À compléter

print("Distribution des périodes:")
print(df_fe['period'].value_counts())

#### Features sur les Montants

In [ ]:
# TODO: Créer les features suivantes sur Amount :
# 1. 'amount_log' : Log transformation (gérer l'asymétrie)
# 2. 'amount_sqrt' : Racine carrée
# 3. 'is_zero_amount' : Indicateur si montant = 0

df_fe['amount_log'] = None # À compléter
df_fe['amount_sqrt'] = None # À compléter
df_fe['is_zero_amount'] = None # À compléter

print(" Features sur montants créées")

In [ ]:
# TODO: Créer un binning du montant
# Catégories suggérées: [0, 10, 50, 100, 500, inf]
# Labels: ['micro', 'small', 'medium', 'large', 'xlarge']

df_fe['amount_bin'] = None # À compléter

print("Distribution des bins:")
print(df_fe['amount_bin'].value_counts())

#### Features d'Interaction

In [ ]:
# TODO: Créer des features d'interaction
# Multiplier Amount avec les 3 features PCA les plus corrélées avec Class
# (identifiées dans l'analyse de corrélation)
# Exemple: df_fe['V1_amount'] = df_fe['V1'] * df_fe['Amount']

# À compléter (au moins 3 interactions)

print(" Features d'interaction créées")

In [ ]:
# TODO: Créer des features d'agrégation sur les V1-V28
# Exemples:
# - Somme des 5 premières features PCA
# - Moyenne des 5 premières features PCA
# - Écart-type des features PCA

# À compléter

print(" Features d'agrégation créées")

#### Features Polynomiales et Ratios

In [ ]:
# TODO: Créer des features polynomiales
# 1. amount_squared = Amount²
# 2. amount_cubed = Amount³
# Justification: Capturer les relations non-linéaires

df_fe['amount_squared'] = None # À compléter
df_fe['amount_cubed'] = None # À compléter

print("Features polynomiales créées")

In [ ]:
# TODO: Créer des features de ratio
# 1. amount_per_hour = Amount / (hour + 1) pour éviter division par 0
# 2. time_amount_ratio = Time / (Amount + 1)
# Justification: Relations entre variables temporelles et montants

df_fe['amount_per_hour'] = None # À compléter
df_fe['time_amount_ratio'] = None # À compléter

print("Features de ratio créées")

In [ ]:
# TODO: Créer des features d'écart à la moyenne
# Pour les 3 features PCA les plus corrélées, calculer:
# deviation_Vi = |Vi - mean(Vi)| / std(Vi)
# Justification: Identifier les valeurs anormales (z-score absolu)

# À compléter pour les 3 features les plus corrélées

print("Features d'écart créées")

#### Validation des Nouvelles Features

In [ ]:
# Compter les nouvelles features créées
original_features = df.shape[1]
new_features_count = df_fe.shape[1] - original_features

print(f"Features originales: {original_features}")
print(f"Nouvelles features créées: {new_features_count}")
print(f"Total features: {df_fe.shape[1]}")

# Lister les nouvelles features
new_features = [col for col in df_fe.columns if col not in df.columns]
print(f"\nNouvelles features: {new_features}")

In [ ]:
# TODO: Analyser la corrélation des nouvelles features avec Class
# Afficher les nouvelles features triées par corrélation absolue

print("Corrélation des nouvelles features avec Class:")
# À compléter

### QUESTION 3
**Justifiez vos choix de Feature Engineering :**

1. Quelles features créées sont les plus prometteuses (corrélation) ?
2. Pourquoi l'encodage cyclique (sin/cos) est-il pertinent pour l'heure ?
3. Quel est l'intérêt métier de créer des interactions Amount × V_i ?

**Vos réponses :**

```
1. ...

2. ...

3. ...
```

---
## PARTIE 2 : MODÉLISATION BASELINE (1h30)
---

### 2.1 Préparation des Données

In [ ]:
# TODO: Séparer X et y
# Attention: Exclure les colonnes catégorielles non encodées (ex: 'period', 'amount_bin')
# ou les encoder avant avec pd.get_dummies()

# Encoder les variables catégorielles si nécessaire
# df_fe_encoded = pd.get_dummies(df_fe, columns=['period', 'amount_bin'], drop_first=True)

X = None # À compléter
y = None # À compléter

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
# TODO: Split Train/Test
# Utiliser stratify=y pour préserver le ratio des classes
# Test size: 20%

X_train, X_test, y_train, y_test = None # À compléter

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nDistribution dans le train:")
print(y_train.value_counts())
print(f"\nDistribution dans le test:")
print(y_test.value_counts())

In [ ]:
# TODO: Scaling
# Choisir entre StandardScaler ou RobustScaler
# RobustScaler est recommandé car il gère mieux les outliers

scaler = None # À compléter

# Fit sur train, transform sur train et test
X_train_scaled = None # À compléter
X_test_scaled = None # À compléter

print(" Scaling effectué")

### QUESTION 4
**Justifiez votre choix de scaler :**

```
J'ai choisi [StandardScaler/RobustScaler] parce que...
```

### 2.2 Modèles Baseline

In [ ]:
# Fonction utilitaire pour évaluer un modèle
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
 """
 Entraîne et évalue un modèle
 
 Returns:
 dict: Dictionnaire avec les métriques
 """
 print(f"\n{'='*60}")
 print(f"ÉVALUATION: {model_name}")
 print(f"{'='*60}")
 
 # TODO: Compléter cette fonction
 # 1. Entraîner le modèle avec fit()
 # 2. Prédire sur le test avec predict() et predict_proba()
 # 3. Calculer les métriques:
 # - Confusion Matrix
 # - Precision, Recall, F1 (precision_score, recall_score, f1_score)
 # - ROC-AUC (roc_auc_score)
 # - PR-AUC (average_precision_score)
 # 4. Afficher les résultats avec print()
 # 5. Retourner un dictionnaire avec les métriques
 
 # À compléter
 
 return {}

print(" Fonction d'évaluation créée")

#### Modèle 1 : Logistic Regression

In [ ]:
# TODO: Créer et évaluer une Logistic Regression
# Utiliser class_weight='balanced' pour gérer le déséquilibre

lr_model = None # À compléter
lr_results = evaluate_model(lr_model, X_train_scaled, X_test_scaled, y_train, y_test, "Logistic Regression")

#### Modèle 2 : Decision Tree

In [ ]:
# TODO: Créer et évaluer un Decision Tree
# Paramètres suggérés: max_depth=10, class_weight='balanced'

dt_model = None # À compléter
dt_results = evaluate_model(dt_model, X_train_scaled, X_test_scaled, y_train, y_test, "Decision Tree")

#### Modèle 3 : Random Forest

In [ ]:
# TODO: Créer et évaluer un Random Forest
# Paramètres suggérés: n_estimators=100, class_weight='balanced'

rf_model = None # À compléter
rf_results = evaluate_model(rf_model, X_train_scaled, X_test_scaled, y_train, y_test, "Random Forest")

#### Modèle 4 : Support Vector Machine (SVM)

In [ ]:
# TODO: Créer et évaluer un SVM
# Paramètres: kernel='rbf', class_weight='balanced', C=1.0
# Note: SVM peut être lent sur de grandes données

from sklearn.svm import SVC

svm_model = None # À compléter
svm_results = evaluate_model(svm_model, X_train_scaled, X_test_scaled, y_train, y_test, "SVM")

#### Modèle 5 : K-Nearest Neighbors (KNN)

In [ ]:
# TODO: Créer et évaluer un KNN
# Paramètres: n_neighbors=5, weights='distance'
# Note: KNN nécessite des données scalées

from sklearn.neighbors import KNeighborsClassifier

knn_model = None # À compléter
knn_results = evaluate_model(knn_model, X_train_scaled, X_test_scaled, y_train, y_test, "KNN")

#### Modèle 6 : XGBoost (Baseline)

In [ ]:
# TODO: Créer et évaluer un XGBoost baseline
# Paramètres: n_estimators=100, max_depth=5, learning_rate=0.1
# scale_pos_weight = nb_negatifs / nb_positifs (pour gérer le déséquilibre)

from xgboost import XGBClassifier

# Calculer scale_pos_weight
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Scale pos weight: {scale_pos_weight:.2f}")

xgb_model = None # À compléter
xgb_results = evaluate_model(xgb_model, X_train_scaled, X_test_scaled, y_train, y_test, "XGBoost")

### 2.3 Comparaison des Modèles Baseline

In [ ]:
# TODO: Créer un tableau comparatif des 6 modèles
# Colonnes: Model, Precision, Recall, F1, ROC-AUC, PR-AUC
# pd.DataFrame([lr_results, dt_results, rf_results, svm_results, knn_results, xgb_results])

comparison_df = None # À compléter

print("\n" + "="*60)
print("COMPARAISON DES MODÈLES BASELINE")
print("="*60)
print(comparison_df)

In [ ]:
# TODO: Visualiser la comparaison avec un bar plot
# Afficher Precision, Recall, F1, PR-AUC pour les 3 modèles



### 2.4 Comparaison des Stratégies de Rééquilibrage

In [ ]:
# TODO: Comparer class_weight='balanced' vs SMOTE
# Prendre le meilleur modèle baseline et tester avec SMOTE
# Formule SMOTE: Génère des exemples synthétiques pour la classe minoritaire
# Pour chaque exemple minoritaire x_i, choisir k voisins et créer:
# x_new = x_i + λ * (x_neighbor - x_i) où λ ∈ [0,1]

from imblearn.over_sampling import SMOTE

# Appliquer SMOTE
smote = SMOTE(random_state=RANDOM_STATE)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"Avant SMOTE: {y_train.value_counts().to_dict()}")
print(f"Après SMOTE: {pd.Series(y_train_smote).value_counts().to_dict()}")

# Entraîner le meilleur modèle avec SMOTE
# À compléter

In [ ]:
# TODO: Comparer les résultats
# Créer un tableau: Stratégie | Precision | Recall | F1 | PR-AUC
# Ligne 1: class_weight='balanced'
# Ligne 2: SMOTE
# Analyser quelle stratégie est la plus efficace



### QUESTION 5
**Analysez les résultats baseline :**

1. Parmi les 6 modèles testés, lequel a le meilleur Recall ? Est-ce suffisant (objectif ≥ 0.85) ?
2. Pourquoi PR-AUC est-il plus pertinent que ROC-AUC ici ?
3. Comparez class_weight vs SMOTE. Quelle stratégie est la plus efficace ?
4. Quel modèle choisiriez-vous pour l'optimisation ? Pourquoi ?

**Vos réponses :**

```
1. ...

2. ...

3. ...

4. ...
```

---
## PARTIE 3 : OPTIMISATION AVANCÉE (2h)
---

**Note** : Cette partie peut prendre du temps (GridSearch = 15-30 min). Commencez par une petite grille pour tester !

### 3.1 Construction d'un Pipeline ML Complet

In [ ]:
# TODO: Créer un Pipeline intégrant:
# 1. Scaling (RobustScaler)
# 2. Modèle (Random Forest - le meilleur baseline)

pipeline = None # À compléter

print(" Pipeline créé")
print(pipeline)

### 3.2 Optimisation Hyperparamètres - GridSearchCV

In [ ]:
# TODO: Définir une grille de paramètres pour Random Forest
# IMPORTANT: Préfixer les paramètres avec 'classifier__' car dans un pipeline
# Commencez PETIT pour tester (2-3 valeurs par paramètre max)
# Exemple:
# param_grid = {
# 'classifier__n_estimators': [100, 200],
# 'classifier__max_depth': [10, None],
# 'classifier__min_samples_split': [2, 5],
# 'classifier__class_weight': ['balanced']
# }

param_grid = {
 # À compléter
}

n_combinations = np.prod([len(v) for v in param_grid.values()])
print(f"Nombre de combinaisons: {n_combinations}")
print(f"Avec cv=5 → {n_combinations * 5} entraînements")

In [ ]:
# TODO: Configurer GridSearchCV
# - cv: StratifiedKFold(n_splits=5)
# - scoring: 'average_precision' (PR-AUC)
# - n_jobs: -1 (parallélisation)
# - verbose: 2 (afficher la progression)

grid_search = None # À compléter

print(" GridSearchCV configuré")

In [ ]:
# TODO: Lancer la recherche (peut prendre plusieurs minutes)
# Utiliser X_train et y_train (NON scalés, le pipeline s'en occupe)

import time
start_time = time.time()

# À compléter: grid_search.fit(...)

end_time = time.time()
print(f"\n Temps d'exécution: {(end_time - start_time) / 60:.2f} minutes")

In [ ]:
# TODO: Afficher les résultats de GridSearch
# - Meilleurs paramètres (grid_search.best_params_)
# - Meilleur score CV (grid_search.best_score_)
# - Évaluer sur le test set (grid_search.best_estimator_)

print("="*60)
print("RÉSULTATS GRIDSEARCH")
print("="*60)

# À compléter

### 3.3 Optimisation avec RandomizedSearchCV

In [ ]:
# TODO: Comparer GridSearchCV vs RandomizedSearchCV
# RandomizedSearchCV teste n_iter combinaisons aléatoires au lieu de toutes
# Avantage: Plus rapide, peut explorer un espace plus large
# Formule: Probabilité de trouver le top 5% en testant n combinaisons aléatoires:
# P = 1 - (0.95)^n
# Exemple: n=20 → P ≈ 64%, n=60 → P ≈ 95%

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Définir des distributions de paramètres (plus larges que GridSearch)
param_distributions = {
 'classifier__n_estimators': randint(50, 300),
 'classifier__max_depth': [5, 10, 15, 20, 25, None],
 'classifier__min_samples_split': randint(2, 20),
 'classifier__min_samples_leaf': randint(1, 10),
 'classifier__max_features': ['sqrt', 'log2', None],
 'classifier__class_weight': ['balanced']
}

# Configurer RandomizedSearchCV
random_search = RandomizedSearchCV(
 pipeline,
 param_distributions,
 n_iter=30, # Tester 30 combinaisons aléatoires
 cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
 scoring='average_precision',
 n_jobs=-1,
 random_state=RANDOM_STATE,
 verbose=2
)

print("RandomizedSearchCV configuré")

In [ ]:
# TODO: Lancer RandomizedSearchCV
import time
start_time = time.time()

# À compléter: random_search.fit(...)

end_time = time.time()
print(f"\nTemps d'exécution: {(end_time - start_time) / 60:.2f} minutes")

# Afficher les résultats
print("\n" + "="*60)
print("RÉSULTATS RANDOMIZEDSEARCH")
print("="*60)
print(f"Meilleurs paramètres: {random_search.best_params_}")
print(f"Meilleur score (CV): {random_search.best_score_:.4f}")

In [ ]:
# TODO: Comparer GridSearch vs RandomizedSearch
# Créer un tableau comparatif:
# Méthode | Meilleur Score | Temps (min) | Nb Combinaisons Testées

comparaison_search = pd.DataFrame({
 'Méthode': ['GridSearchCV', 'RandomizedSearchCV'],
 'Meilleur_Score': [None, None], # À compléter
 'Temps_min': [None, None], # À compléter
 'Nb_Combinaisons': [None, 30]
})

print("\n" + "="*60)
print("COMPARAISON DES MÉTHODES D'OPTIMISATION")
print("="*60)
print(comparaison_search)

### 3.4 Optimisation de XGBoost

In [ ]:
# TODO: Optimiser XGBoost avec RandomizedSearchCV
# Créer un pipeline avec XGBoost
# Paramètres à optimiser:
# - n_estimators: [50, 100, 200, 300]
# - max_depth: [3, 5, 7, 9]
# - learning_rate: [0.01, 0.05, 0.1, 0.3]
# - subsample: [0.6, 0.8, 1.0]
# - colsample_bytree: [0.6, 0.8, 1.0]
# - scale_pos_weight: calculé automatiquement

pipeline_xgb = Pipeline([
 ('scaler', RobustScaler()),
 ('classifier', XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss'))
])

# À compléter: définir param_distributions et lancer RandomizedSearchCV


### QUESTION 6
**Analysez l'optimisation :**

1. Quels hyperparamètres ont le plus d'impact sur la performance ?
2. Le gain de performance justifie-t-il le temps de calcul ?
3. GridSearch vs RandomizedSearch: quelle méthode est la plus efficace ?
4. XGBoost optimisé vs Random Forest optimisé: lequel est meilleur ?
5. Atteignez-vous l'objectif de Recall ≥ 0.85 ?

**Vos réponses :**

```
1. ...

2. ...

3. ...

4. ...

5. ...
```

---
## PARTIE 4 : ANALYSE & DIAGNOSTIC (1h)
---

### 4.1 Feature Importance

In [ ]:
# TODO: Extraire l'importance des features du meilleur modèle

best_model = None # À compléter
feature_importances = None # À compléter

# Créer un DataFrame et afficher le Top 15
# À compléter

In [ ]:
# TODO: Visualiser le Top 15 avec un bar plot horizontal



### 4.1bis Permutation Importance

In [ ]:
# TODO: Calculer la Permutation Importance
# Principe: Mélanger aléatoirement une feature et mesurer la baisse de performance
# Formule: PI(f) = Score_original - Score_après_permutation(f)
# Plus PI est élevé, plus la feature est importante

from sklearn.inspection import permutation_importance

# Calculer la permutation importance
perm_importance = permutation_importance(
 best_model,
 X_test,
 y_test,
 n_repeats=10,
 random_state=RANDOM_STATE,
 scoring='average_precision'
)

# Créer un DataFrame
perm_imp_df = pd.DataFrame({
 'feature': X_train.columns,
 'importance_mean': perm_importance.importances_mean,
 'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

print("Top 15 Features - Permutation Importance:")
print(perm_imp_df.head(15))

In [ ]:
# TODO: Comparer MDI (Mean Decrease Impurity) vs Permutation Importance
# Créer un graphique côte à côte
# MDI peut être biaisé vers les features à haute cardinalité
# Permutation Importance est plus fiable mais plus coûteux en calcul



### 4.1ter SHAP Values (Explicabilité Avancée)

In [ ]:
# TODO: Calculer les SHAP values
# SHAP (SHapley Additive exPlanations) basé sur la théorie des jeux
# Formule de Shapley: φ_i = Σ [|S|!(n-|S|-1)!/n!] × [f(S∪{i}) - f(S)]
# où S sont tous les sous-ensembles de features sans i
# Interprétation: Contribution marginale moyenne de chaque feature

import shap

# Créer l'explainer (TreeExplainer pour Random Forest/XGBoost)
explainer = shap.TreeExplainer(best_model.named_steps['classifier'])

# Calculer SHAP values sur un échantillon (100 premières lignes du test)
X_test_sample = X_test.iloc[:100]
shap_values = explainer.shap_values(X_test_sample)

print("SHAP values calculés")
print(f"Shape: {shap_values[1].shape if isinstance(shap_values, list) else shap_values.shape}")

In [ ]:
# TODO: Visualiser le summary plot SHAP
# Ce graphique montre:
# - L'importance de chaque feature (axe Y)
# - L'impact sur la prédiction (axe X)
# - La valeur de la feature (couleur: rouge=élevé, bleu=faible)

# Pour classification binaire, prendre shap_values[1] (classe positive)
shap_values_plot = shap_values[1] if isinstance(shap_values, list) else shap_values

shap.summary_plot(shap_values_plot, X_test_sample, plot_type="dot", show=False)
plt.tight_layout()
plt.show()

In [ ]:
# TODO: Expliquer une prédiction individuelle
# Choisir une fraude correctement détectée et analyser pourquoi
# SHAP force plot montre la contribution de chaque feature à la prédiction

# Trouver un exemple de fraude
fraud_idx = y_test[y_test == 1].index[0]
fraud_sample_idx = X_test.index.get_loc(fraud_idx)

if fraud_sample_idx < 100: # Si dans notre échantillon
 print(f"Analyse de la transaction {fraud_idx} (Fraude)")
 print(f"Probabilité prédite: {best_model.predict_proba(X_test.iloc[[fraud_sample_idx]])[:,1][0]:.4f}")
 
 # Force plot
 shap.force_plot(
 explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
 shap_values_plot[fraud_sample_idx],
 X_test_sample.iloc[fraud_sample_idx],
 matplotlib=True,
 show=False
 )
 plt.tight_layout()
 plt.show()

### QUESTION 7
**Interprétez les Feature Importances :**

1. Les features créées (Feature Engineering) sont-elles utiles ?
2. Y a-t-il des surprises (features inattendues importantes) ?
3. MDI vs Permutation Importance: quelles différences observez-vous ?
4. SHAP: quelles features contribuent le plus aux prédictions de fraude ?
5. Pourrait-on simplifier le modèle en retirant des features peu importantes ?

**Vos réponses :**

```
1. ...

2. ...

3. ...

4. ...

5. ...
```

### 4.2 Courbes ROC et Precision-Recall

In [ ]:
# TODO: Tracer les courbes ROC et Precision-Recall côte à côte
# 1. Prédire les probabilités avec predict_proba()
# 2. Calculer les courbes avec roc_curve() et precision_recall_curve()
# 3. Visualiser avec plt.subplot(1, 2, 1) et plt.subplot(1, 2, 2)



### 4.3 Learning Curves

In [ ]:
# TODO: Tracer les Learning Curves
# Utiliser learning_curve de sklearn
# Paramètres: cv=5, scoring='average_precision', train_sizes=np.linspace(0.1, 1.0, 10)



### 4.3bis Calibration des Probabilités

In [ ]:
# TODO: Vérifier la calibration du modèle
# Un modèle bien calibré: si proba=0.8, alors 80% des prédictions sont correctes
# Méthode: Tracer la courbe de calibration (reliability diagram)
# Axe X: Probabilité prédite (bins)
# Axe Y: Fraction réelle de positifs dans chaque bin
# Ligne diagonale = calibration parfaite

from sklearn.calibration import calibration_curve

# Calculer la courbe de calibration
fraction_of_positives, mean_predicted_value = calibration_curve(
 y_test,
 y_proba_optimized,
 n_bins=10,
 strategy='uniform'
)

# Visualiser
plt.figure(figsize=(10, 6))
plt.plot(mean_predicted_value, fraction_of_positives, 's-', label='Modèle')
plt.plot([0, 1], [0, 1], 'k--', label='Calibration parfaite')
plt.xlabel('Probabilité prédite moyenne')
plt.ylabel('Fraction de positifs')
plt.title('Courbe de Calibration')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Interprétation:")
print("- Si proche de la diagonale: modèle bien calibré")
print("- Si au-dessus: modèle sous-estime les probabilités")
print("- Si en-dessous: modèle surestime les probabilités")

In [ ]:
# TODO: Appliquer la calibration si nécessaire
# Méthode: CalibratedClassifierCV avec Platt Scaling ou Isotonic Regression
# Platt Scaling: Ajuste une régression logistique sur les probabilités
# Formule: P_calibrated = 1 / (1 + exp(A × log(p/(1-p)) + B))
# Isotonic Regression: Ajuste une fonction monotone non-paramétrique

from sklearn.calibration import CalibratedClassifierCV

# Calibrer le modèle
calibrated_model = CalibratedClassifierCV(
 best_model,
 method='sigmoid', # Platt Scaling
 cv=5
)

# Entraîner sur le train
calibrated_model.fit(X_train, y_train)

# Prédire sur le test
y_proba_calibrated = calibrated_model.predict_proba(X_test)[:, 1]

# Comparer les performances
print("\nComparaison avant/après calibration:")
print(f"PR-AUC avant: {average_precision_score(y_test, y_proba_optimized):.4f}")
print(f"PR-AUC après: {average_precision_score(y_test, y_proba_calibrated):.4f}")

### QUESTION 8
**Diagnostiquez le modèle :**

1. Le modèle est-il en overfitting, underfitting ou bon fit ?
2. Le modèle bénéficierait-il de plus de données ?
3. Quelles actions recommanderiez-vous pour améliorer le modèle ?

**Vos réponses :**

```
1. ...

2. ...

3. ...
```

### 4.4 Optimisation du Seuil de Décision

In [ ]:
# TODO: Trouver le seuil optimal maximisant le F1-Score
# 1. Calculer precision/recall pour tous les seuils avec precision_recall_curve()
# 2. Calculer F1 pour chaque seuil: F1 = 2 * (P * R) / (P + R)
# 3. Trouver le seuil qui maximise F1
# 4. Comparer les métriques avec seuil 0.5 vs seuil optimal



### QUESTION 9
**Analysez l'optimisation du seuil :**

1. Quel est l'impact sur Precision et Recall ?
2. Quel seuil recommanderiez-vous en production ? Pourquoi ?
3. Comment choisir entre privilégier Precision ou Recall selon le contexte métier ?

**Vos réponses :**

```
1. ...

2. ...

3. ...
```

---
## PARTIE 5 : DÉPLOIEMENT & PRODUCTION (1h)
---

### 5.1 Validation Temporelle (TimeSeriesSplit)

In [ ]:
# TODO: Valider le modèle avec TimeSeriesSplit
# Comparer avec StratifiedKFold



### 5.2 Sérialisation du Modèle

In [ ]:
# TODO: Sauvegarder le meilleur modèle (pipeline complet)

import joblib
import os

# Créer le dossier models s'il n'existe pas
os.makedirs('../models', exist_ok=True)

# À compléter

In [ ]:
# TODO: Sauvegarder les métadonnées
# Inclure: version, date, paramètres, scores, seuil optimal, features, etc.

metadata = {
 'model_version': '1.0',
 'train_date': '2025-12-14',
 'random_state': RANDOM_STATE,
 # À compléter
}

# À compléter: joblib.dump(metadata, ...)

### 5.3 Test de Chargement et Prédiction

In [ ]:
# TODO: Charger le modèle sauvegardé et tester une prédiction
# 1. Charger avec joblib.load()
# 2. Prédire sur un échantillon du test
# 3. Afficher les résultats



### 5.4 Création d'une API de Prédiction (Flask)

In [ ]:
# TODO: Créer un script Flask pour l'API
# Structure:
# - Endpoint POST /predict
# - Input: JSON avec les features d'une transaction
# - Output: JSON avec {"is_fraud": bool, "probability": float, "risk_level": str}

# Créer le fichier api_fraud_detection.py
api_code = '''
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np

app = Flask(__name__)

# Charger le modèle au démarrage
model = joblib.load('models/fraud_detector_v1.joblib')
metadata = joblib.load('models/metadata_v1.joblib')

@app.route('/predict', methods=['POST'])
def predict():
 """
 Endpoint de prédiction
 Input JSON: {"Time": float, "V1": float, ..., "Amount": float}
 Output JSON: {"is_fraud": bool, "probability": float, "risk_level": str}
 """
 try:
 # Récupérer les données
 data = request.get_json()
 
 # Convertir en DataFrame
 df = pd.DataFrame([data])
 
 # Prédire
 proba = model.predict_proba(df)[:, 1][0]
 threshold = metadata.get('optimal_threshold', 0.5)
 is_fraud = proba >= threshold
 
 # Déterminer le niveau de risque
 if proba < 0.3:
 risk_level = 'low'
 elif proba < 0.6:
 risk_level = 'medium'
 elif proba < 0.85:
 risk_level = 'high'
 else:
 risk_level = 'critical'
 
 # Retourner la réponse
 return jsonify({
 'is_fraud': bool(is_fraud),
 'probability': float(proba),
 'risk_level': risk_level,
 'threshold_used': float(threshold)
 })
 
 except Exception as e:
 return jsonify({'error': str(e)}), 400

@app.route('/health', methods=['GET'])
def health():
 """Endpoint de santé"""
 return jsonify({'status': 'ok', 'model_version': metadata.get('model_version', 'unknown')})

if __name__ == '__main__':
 app.run(host='0.0.0.0', port=5000, debug=False)
'''

# Sauvegarder le fichier
with open('../api_fraud_detection.py', 'w') as f:
 f.write(api_code)

print("API Flask créée: ../api_fraud_detection.py")
print("\nPour lancer l'API:")
print(" python api_fraud_detection.py")
print("\nPour tester:")
print(" curl -X POST http://localhost:5000/predict -H 'Content-Type: application/json' -d '{...}'")

### 5.5 Tests Unitaires du Modèle

In [ ]:
# TODO: Créer des tests unitaires pour valider le modèle
# Tests à implémenter:
# 1. Test de chargement du modèle
# 2. Test de prédiction sur des données valides
# 3. Test de robustesse (valeurs manquantes, outliers)
# 4. Test de performance (temps de prédiction < 100ms)

import unittest
import time

class TestFraudDetector(unittest.TestCase):
 
 @classmethod
 def setUpClass(cls):
 """Charger le modèle une fois pour tous les tests"""
 cls.model = joblib.load('../models/fraud_detector_v1.joblib')
 cls.sample_data = X_test.iloc[:10]
 
 def test_model_loading(self):
 """Test: Le modèle se charge correctement"""
 self.assertIsNotNone(self.model)
 print("Test chargement: OK")
 
 def test_prediction_shape(self):
 """Test: Les prédictions ont la bonne forme"""
 predictions = self.model.predict(self.sample_data)
 self.assertEqual(len(predictions), len(self.sample_data))
 print("Test shape prédictions: OK")
 
 def test_prediction_values(self):
 """Test: Les prédictions sont dans [0, 1]"""
 probas = self.model.predict_proba(self.sample_data)[:, 1]
 self.assertTrue(all(0 <= p <= 1 for p in probas))
 print("Test valeurs prédictions: OK")
 
 def test_prediction_time(self):
 """Test: Temps de prédiction < 100ms pour 10 transactions"""
 start = time.time()
 _ = self.model.predict(self.sample_data)
 elapsed = (time.time() - start) * 1000 # en ms
 self.assertLess(elapsed, 100)
 print(f"Test temps prédiction: {elapsed:.2f}ms < 100ms OK")

# Exécuter les tests
if __name__ == '__main__':
 suite = unittest.TestLoader().loadTestsFromTestCase(TestFraudDetector)
 runner = unittest.TextTestRunner(verbosity=2)
 result = runner.run(suite)
 
 print(f"\nTests exécutés: {result.testsRun}")
 print(f"Succès: {result.testsRun - len(result.failures) - len(result.errors)}")
 print(f"Échecs: {len(result.failures)}")
 print(f"Erreurs: {len(result.errors)}")

---
## SYNTHÈSE FINALE
---

### QUESTION 10 - BILAN GLOBAL
**Rédigez une synthèse de votre travail (10-15 lignes) :**

1. **Performance finale** : Avez-vous atteint les objectifs (Recall ≥ 0.85, Precision maximale) ?
2. **Feature Engineering** : Quelles features créées ont été les plus utiles ?
3. **Optimisation** : Quel a été l'impact de GridSearch et de l'ajustement du seuil ?
4. **Déploiement** : Le modèle est-il prêt pour la production ? Quelles précautions ?
5. **Améliorations futures** : Que pourriez-vous faire pour améliorer encore le modèle ?

**Votre synthèse :**

```
...
```

---
## BONUS (Optionnel)
---

Si vous avez terminé en avance, voici quelques pistes pour aller plus loin :

### Bonus 1 : XGBoost

In [ ]:
# TODO: Tester XGBoost avec optimisation des hyperparamètres
# Paramètre clé: scale_pos_weight (ratio des classes)

from xgboost import XGBClassifier

# À compléter

### Bonus 2 : SHAP Values

In [ ]:
# TODO: Utiliser SHAP pour expliquer les prédictions
# import shap

# À compléter

### Bonus 3 : Voting Classifier

In [ ]:
# TODO: Combiner plusieurs modèles avec VotingClassifier
# from sklearn.ensemble import VotingClassifier

# À compléter

---
## FIN DU TP

**Félicitations ! **

Vous avez complété un pipeline ML complet de détection de fraude, de l'exploration à la production.

**N'oubliez pas de :**
- Sauvegarder ce notebook
- Vérifier que tous les fichiers sont créés (modèle, métadonnées)
- Répondre à toutes les 10 questions
- Compléter les modules Python si demandé (`utils/predict.py`)

---

**Master 1 Data Engineering - YNOV Montpellier** 
**Date** : Décembre 2025

---